In [1]:
%pylab inline

import math
import numpy as np
from scipy.sparse.linalg import inv
#from numpy.linalg import inv
import scipy.sparse as sps
import scipy.sparse.linalg
from scipy import integrate
import sys
import matplotlib.pyplot as plt
sys.path.append('../../src/')
from pylab import *

import parameters as pam
import lattice as lat
import variational_space as vs
import hamiltonian as ham
import basis_change as basis
import lanczos

ed = 0
As = np.arange(0.0, 12.01, 0.5)
eps = np.arange(2.75, 2.751, 0.1)
tpp = 0.55
tpd = 0.5

pds = 1.5
pdp = 0.7
#pds = 0.00001
#pdp = 0.00001
pps = 1.0
ppp = 0.3

B = 0.15
C = 0.58
Upp = 0
Norb = 7
Mc = 25
eta = 0.001
symmetries = ['1A1']#,'3B1']

Ms = ['-b','-r','-g','-m','-c','-k','-y','--b','--r','--g','--m','--c','--k','--y',\
      '-.b','-.r','-.g','-.m','-.c','-.k','-.y',':b',':r',':g',':m',':c',':k',':y']

def write_wpeak(fname,ep,tpd,wpeak):
    #"a" - Append - will append to the end of the file
    #"w" - Write - will overwrite any existing content
    f = open('./Norb7/data_lowpeak/'+fname,'a',1) 
    f.write('{:.6e}\t{:.6e}\t{:.6e}\n'.format(ep,tpd,wpeak))
    
def write_intensity(fname,A,ep,tpd,intensity):
    #"a" - Append - will append to the end of the file
    #"w" - Write - will overwrite any existing content
    f = open('./Norb7/data_lowpeak/'+fname,'a',1) 
    f.write('{:.6e}\t{:.6e}\t{:.6e}\t{:.6e}\n'.format(A,ep,tpd,intensity))
    
def getAw_peak_pos_weight(ff,Aw):  
    '''
    find the position and weight of lowest peak of Aw
    '''    
    w_bottom = min(Aw[:,0])
    wid = find(Aw[:,0]==w_bottom)
    
    # go through the regime with Aw=0 (numerically ~1.e-6)
    while Aw[wid,1]<1.e-4:
        wid += 1
    print 'Aw<1.e-4 until ', Aw[wid,0]

    # go up until the peak:
    while Aw[wid+1,1]>Aw[wid,1]:
        wid += 1
    print 'lowest peak at w= ', Aw[wid,0]
    write_wpeak(ff,ep,tpd,float(Aw[wid,0]))
        
def getAw_peak_lowest(ff, Awdata):  
    '''
    find the position and weight of lowest peak of Aw, which might be highest
    '''    
    w_vals = Awdata[:,0]
    Aw     = Awdata[:,1]
    
    w_idx = 0
    # go through the regime with Aw=0 (numerically ~1.e-6)
    while Aw[w_idx]<1.e-3:
        w_idx += 1
    print 'Aw < 1.e-3 until ', w_vals[w_idx]

    # go up until the peak:
    while Aw[w_idx+1]>Aw[w_idx]:
        w_idx += 1
    w_peak = w_vals[w_idx]
    print 'lowest peak at w = ', w_peak
    
    # find the area below the whole peak, namely the peak weight
    # ==========================================================
    # 1. first find the peak's w-range: [w_min, w_max]
    wid = w_idx
    while Aw[wid]>1.e-3:
        #print w_vals[wid], Aw[wid]
        if Aw[wid-1]>Aw[wid]:
            break
        wid -= 1
    w_min = wid
    
    wid = w_idx
    while Aw[wid]>1.e-3:
        #print w_vals[wid], Aw[wid]
        if Aw[wid+1]>Aw[wid]:
            break
        wid += 1
    w_max = wid
    
    print 'lowest peak w-range = [', w_vals[w_min], w_vals[w_max], ']'
    
    # 2. Simpson's rule
    weight = integrate.simps(Aw[w_min:w_max], w_vals[w_min:w_max])
    print 'lowest peak, weight = ', w_peak, '  ', weight
    
    write_wpeak(ff,ep,tpd,w_peak)

def getAw_peak_lowest_intensity(ff, Awdata, A, ep):  
    '''
    find the intensity of lowest peak of Aw, which might be highest
    '''    
    w_vals = Awdata[:,0]
    Aw     = Awdata[:,1]
    
    w_idx = 0
    # go through the regime with Aw=0 (numerically ~1.e-6)
    while Aw[w_idx]<1.e-3:
        w_idx += 1
    print 'Aw < 1.e-3 until ', w_vals[w_idx]

    # go up until the peak:
    while Aw[w_idx+1]>Aw[w_idx]:
        w_idx += 1
    print 'lowest peak intensity = ', Aw[w_idx]
    
    write_intensity(ff,A,ep,tpd,Aw[w_idx])

def getAw_peak_highest(ff, Awdata):  
    '''
    find the position and weight of lowest peak of Aw, which might be highest
    '''    
    w_vals = Awdata[:,0]
    Aw     = Awdata[:,1] 

    w_idx = np.argmax(Aw)
    print 'highest peak index',w_idx
    w_peak = w_vals[w_idx]
    print 'highest peak at w = ', w_peak
    
    # find the area below the whole peak, namely the peak weight
    # ==========================================================
    # 1. first find the peak's w-range: [w_min, w_max]
    wid = w_idx
    while Aw[wid]>1.e-3:
        #print w_vals[wid], Aw[wid]
        if Aw[wid-1]>Aw[wid]:
            break
        wid -= 1
    w_min = wid
    
    wid = w_idx
    while Aw[wid]>1.e-3:
        #print w_vals[wid], Aw[wid]
        if Aw[wid+1]>Aw[wid]:
            break
        wid += 1
    w_max = wid
    
    print 'highest peak w-range = [', w_vals[w_min], w_vals[w_max], ']'
    
    # 2. Simpson's rule
    weight = integrate.simps(Aw[w_min:w_max], w_vals[w_min:w_max])
    print 'highest peak, weight = ', w_peak, '  ', weight
    
    '''
    # find the eigenvalue D[n] nearest to w_peak so that its index n
    # leads to weight = tab[n]; Note that this weight is actually for 
    # the single peak instead of the area below the whole peak
    tmp = []
    for n in range(len(D)):
        tmp.append(abs(D[n]-w_peak))
        
    idx = tmp.index(min(tmp))
    weight = tab[idx]
    assert(weight>=0.0 and weight<=1.0)
    '''
    write_wpeak(ff,ep,tpd,w_peak)

for A in As:
    for ep in eps:
        print 'ep=',ep
        Nsym = len(symmetries)
        clf()
        for i in range(0,Nsym):
            sym = symmetries[i]
            fname = 'ep'+str(ep)+'_tpd'+str(tpd)+'_tpp'+str(tpp)+'_A'+str(A)+'_B'+str(B)+'_C'+str(C) \
                          +'_Upp'+str(Upp)+'_Mc'+str(Mc)+'_Norb'+str(Norb)+'_eta'+str(eta)
            ff = 'lowpeak_intensity_Norb7_tpd'+str(tpd)+'_tpp'+str(tpp)+'_B'+str(B)+'_C'+str(C) \
                          +'_Upp'+str(Upp)+'_Mc'+str(Mc)+'_eta'+str(eta)+'_1A1.txt'

            a = loadtxt('./Norb7/data_Aw/'+fname+'_'+sym+'.txt',skiprows=0)
            #getAw_peak_lowest(ff,a)
            getAw_peak_lowest_intensity(ff,a,A,ep)

Populating the interactive namespace from numpy and matplotlib
Cu_orbs =  ['d3z2r2', 'dx2y2', 'dxy', 'dxz', 'dyz']
O1_orbs =  ['px']
O2_orbs =  ['py']
turn on interactions for symmetries =  ['ALL']
compute A(w) for symmetries =  []
ep= 2.75
Aw < 1.e-3 until  -0.708
lowest peak intensity =  73.05544
ep= 2.75
Aw < 1.e-3 until  -0.541
lowest peak intensity =  46.88776
ep= 2.75
Aw < 1.e-3 until  -0.425
lowest peak intensity =  27.26575
ep= 2.75
Aw < 1.e-3 until  -0.345
lowest peak intensity =  21.26793
ep= 2.75
Aw < 1.e-3 until  -0.287
lowest peak intensity =  14.12035
ep= 2.75
Aw < 1.e-3 until  -0.245
lowest peak intensity =  9.7454
ep= 2.75
Aw < 1.e-3 until  -0.213
lowest peak intensity =  8.829413
ep= 2.75
Aw < 1.e-3 until  -0.188
lowest peak intensity =  6.782815
ep= 2.75
Aw < 1.e-3 until  -0.168
lowest peak intensity =  4.83844
ep= 2.75
Aw < 1.e-3 until  -0.152
lowest peak intensity =  4.702219
ep= 2.75
Aw < 1.e-3 until  -0.139
lowest peak intensity =  3.893884
ep= 2.75
Aw < 1.e-3 unt